In [11]:
'''
>>>> Unfortunately, the Python standard library doesn't provide a built-in 
method to retrieve the user who changed a file. To achieve this, 
you can use a third-party library called pywin32 to access Windows 
file auditing features. However, this requires enabling file auditing 
on the folder you want to monitor. Please follow these steps:

>>>> Enable file auditing on the folder:
        > Right-click the folder, and select Properties.
        > Go to the Security tab, click Advanced.
        > Switch to the Auditing tab, and click Continue.
        > Click Add, and then click Select a principal.
        > Type "Everyone" in the text box, and click OK.
        > Check the boxes for "Success" and "Failed" under "Write" and "Delete" permissions.
        > Click OK, and then click Apply.
        > Close the folder properties.

>>>> Install the pywin32 library by running pip install pywin32 
in your terminal or command prompt.

>>>> Use the following modified function to check for recent 
changes and the user who made the changes:
'''
#***********************************************
import os
import datetime
import win32security
import win32con
import pywintypes
import win32evtlog
#***********************************************
#***********************************************
def get_file_owner(file_path):
    # Get the security descriptor for the file
    sd = win32security.GetFileSecurity(file_path, win32security.OWNER_SECURITY_INFORMATION)

    # Get the owner's SID
    owner_sid = sd.GetSecurityDescriptorOwner()

    # Convert the owner's SID to a human-readable name
    name, domain, _ = win32security.LookupAccountSid(None, owner_sid)

    return f"{domain}\\{name}"

#***********************************************
#***********************************************
def get_last_write_event(file_path):
    logtype = "Security"
    handle = win32evtlog.OpenEventLog(None, logtype)
    flags = win32evtlog.EVENTLOG_BACKWARDS_READ | win32evtlog.EVENTLOG_SEQUENTIAL_READ
    events = win32evtlog.ReadEventLog(handle, flags, 0)

    for event in events:
        if event.EventID == 4663:
            object_name = win32evtlogutil.SafeFormatMessage(event, logtype, stringInsertsOnly=True).strip()
            if object_name.lower() == file_path.lower():
                return event

    win32evtlog.CloseEventLog(handle)
    return None
#***********************************************
#***********************************************
def check_recent_changes(directory, minutes):
    recent_changes = []
    threshold = datetime.datetime.now() - datetime.timedelta(minutes=minutes)

    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_mtime = datetime.datetime.fromtimestamp(os.path.getmtime(file_path))

            if file_mtime > threshold:
                owner = get_file_owner(file_path)
                last_write_event = get_last_write_event(file_path)

                if last_write_event:
                    event_time = last_write_event.TimeGenerated
                    user_sid = last_write_event.StringInserts[1]
                    user_name, _, _ = win32security.LookupAccountSid(None, win32security.ConvertStringSidToSid(user_sid))
                else:
                    event_time = None
                    user_name = "Unknown"

                recent_changes.append((file, file_mtime, event_time, user_name, owner))

    return recent_changes
#***********************************************
#***********************************************
def check_recent_change(file_path, minutes):
    threshold = datetime.datetime.now() - datetime.timedelta(minutes=minutes)
    file_mtime = datetime.datetime.fromtimestamp(os.path.getmtime(file_path))

    if file_mtime > threshold:
        file_owner = get_file_owner(file_path)
        return True, file_mtime, file_owner
    else:
        return False, file_mtime, None
#***********************************************
#***********************************************
def finalFileChangeCheck(directory_to_check, time_period_in_minutes):
    #directory_to_check = r"C:\\Users\\nchar\\Documents\\Github\\python\\WindowsPython"  # Replace with the path of the directory you want to check
    #time_period_in_minutes = 10

    changes = check_recent_changes(directory_to_check, time_period_in_minutes)
    if changes:
        print(f"Files changed in the last {time_period_in_minutes} minutes:")
        for file, mtime, event_time, user_name, owner in changes:
            print(f"File: {file}")
            print(f"Modified at: {mtime}")
            print(f"Event time: {event_time}")
            print(f"User who made the change: {user_name}")
            print(f"Owner: {owner}")
            print("-" * 50)
    else:
        print(f"No changes in the last {time_period_in_minutes} minutes.")
#***********************************************
#***********************************************


In [12]:
directory_to_check = r"C:\\Users\\nchar\\Documents\\Github\\python\\WindowsPython"  # Replace with the path of the directory you want to check
time_period_in_minutes = 10
finalFileChangeCheck(directory_to_check, time_period_in_minutes)

Files changed in the last 10 minutes:
File: FindFileChangesAdvanced.ipynb
Modified at: 2023-04-29 20:25:30.852602
Event time: None
User who made the change: Unknown
Owner: BUILTIN\Administrators
--------------------------------------------------
File: Untitled.ipynb
Modified at: 2023-04-29 20:18:44.874564
Event time: None
User who made the change: Unknown
Owner: BUILTIN\Administrators
--------------------------------------------------
File: FindFileChangesAdvanced-checkpoint.ipynb
Modified at: 2023-04-29 20:25:30.852602
Event time: None
User who made the change: Unknown
Owner: BUILTIN\Administrators
--------------------------------------------------
